# Generierung synthetischer Prompts für Gerichtsurteile und Pressemitteilungen

Dieses Notebook verwendet die Claude API, um synthetische Prompts für die Pressemitteilungen - Gerichtsurteilen Paare zu erstellen.

## 1. Setup und Datensatz-Laden

Zuerst importieren wir die notwendigen Bibliotheken und laden den bereinigten Datensatz aus `bereinigung.ipynb`.

In [17]:
import pandas as pd
import numpy as np
import os
import requests
import json
import time
from tqdm.auto import tqdm
import anthropic
from pathlib import Path
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')


In [18]:
# Lade den bereinigten Datensatz
try:
    df = pd.read_csv('../data/interim/cleaned.csv')
    print(f"Bereinigter Datensatz mit {len(df)} Einträgen geladen")
except FileNotFoundError:
    print("Bereinigter Datensatz nicht gefunden.")

# Stichprobe des Datensatzes für Tests (für Produktionseinsatz auskommentiert)
# df = df.sample(5, random_state=42)

Bereinigter Datensatz mit 6432 Einträgen geladen


## 2. Setup Claude API-Zugang

Als nächstes richten wir die Verbindung zur Claude API ein. Wir verwenden den Anthropic Python-Client.

In [30]:
# Load the .env file with API key
env_path = '/Users/sebastiannagl/Desktop/Code/CourtPressGER/.env'
print(f"Looking for .env file at: {env_path}")

# Explicitly read the .env file content
try:
    with open(env_path, 'r') as f:
        for line in f:
            if line.strip() and '=' in line:
                key, value = line.strip().split('=', 1)
                os.environ[key] = value
                print(f"Set environment variable: {key}")
except Exception as e:
    print(f"Error reading .env file: {e}")

# Backup approach using dotenv
load_dotenv(env_path)

# Get the API key
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
print(f"API key found: {'Yes' if ANTHROPIC_API_KEY else 'No'}")

if not ANTHROPIC_API_KEY:
    print("Kein API-Schlüssel in der Umgebung gefunden.")
    ANTHROPIC_API_KEY = input("Bitte gib deinen Anthropic API-Schlüssel ein: ")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
else:
    print("API-Schlüssel aus der Umgebung geladen!")

# Initialize the Claude client with the API key as a direct parameter
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Test the API connection
try:
    # Make a simple query to test the connection
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=100,
        messages=[
            {"role": "user", "content": "Hallo, Claude! Bitte antworte mit einer sehr kurzen Begrüßung."}
        ]
    )
    print("API-Verbindung erfolgreich!")
    print(f"Claude sagt: {response.content[0].text}")
    
    # Get available models - with proper error handling
    try:
        print("Currently available models:")
        models = client.models.list()
        # Handle both possible response formats
        if hasattr(models, 'data'):
            for model in models.data:
                if hasattr(model, 'id') and hasattr(model, 'display_name'):
                    print(f"- {model.id}: {model.display_name}")
                else:
                    print(f"- {model}")
        else:
            for model in models:
                print(f"- {model.get('id', 'unknown')}")
    except Exception as e:
        print(f"Fehler beim Abrufen der Modelle: {e}")
except Exception as e:
    print(f"Fehler bei der Verbindung zur Claude API: {e}")



Looking for .env file at: /Users/sebastiannagl/Desktop/Code/CourtPressGER/.env
Set environment variable: ANTHROPIC_API_KEY
API key found: Yes
API-Schlüssel aus der Umgebung geladen!
API-Verbindung erfolgreich!
Claude sagt: Hallo! Schön, von dir zu hören.
Currently available models:
- claude-3-7-sonnet-20250219: Claude 3.7 Sonnet
- claude-3-5-sonnet-20241022: Claude 3.5 Sonnet (New)
- claude-3-5-haiku-20241022: Claude 3.5 Haiku
- claude-3-5-sonnet-20240620: Claude 3.5 Sonnet (Old)
- claude-3-haiku-20240307: Claude 3 Haiku
- claude-3-opus-20240229: Claude 3 Opus


## 3. Hilfsfunktionen für die Generierung synthetischer Prompts

Wir erstellen Hilfsfunktionen, um synthetische Prompts aus den Gerichtsurteilen und Pressemitteilungen zu generieren.
Die Implementierung berücksichtigt die Rate-Limits der Claude API (50 Anfragen/Minute, 20.000 Input-Tokens/Minute, 8.000 Output-Tokens/Minute).

In [44]:
class RateLimiter:
    """
    Klasse zur Verwaltung der Rate-Limits für die Claude API.
    - 50 Anfragen pro Minute
    - 20.000 Input-Tokens pro Minute
    - 8.000 Output-Tokens pro Minute
    """
    def __init__(self, requests_per_minute=50, input_tokens_per_minute=20000, output_tokens_per_minute=8000):
        self.requests_per_minute = requests_per_minute
        self.input_tokens_per_minute = input_tokens_per_minute
        self.output_tokens_per_minute = output_tokens_per_minute
        
        # Trackers für die letzten 60 Sekunden (Zeitfenster von 1 Minute)
        self.request_timestamps = []
        self.input_token_usage = []  # [timestamp, tokens]
        self.output_token_usage = []  # [timestamp, tokens]
        
    def wait_if_needed(self, input_tokens_estimate, output_tokens_estimate=500):
        """
        Wartet, wenn nötig, um die Rate-Limits einzuhalten.
        Schätzt die Anzahl der benötigten Tokens und wartet, bis genügend Kapazität verfügbar ist.
        """
        current_time = time.time()
        
        # Bereinige abgelaufene Einträge (älter als 60 Sekunden)
        self._clean_expired_entries(current_time)
        
        # Berechne aktuelle Nutzung
        recent_requests = len(self.request_timestamps)
        recent_input_tokens = sum(tokens for _, tokens in self.input_token_usage)
        recent_output_tokens = sum(tokens for _, tokens in self.output_token_usage)
        
        wait_time = 0
        
        # Prüfe Request-Limit
        if recent_requests >= self.requests_per_minute:
            oldest_request = self.request_timestamps[0]
            wait_time = max(wait_time, oldest_request + 60 - current_time)
        
        # Prüfe Input-Token-Limit
        if recent_input_tokens + input_tokens_estimate > self.input_tokens_per_minute:
            wait_time_input = self._calculate_token_wait_time(self.input_token_usage, 
                                                            input_tokens_estimate, 
                                                            self.input_tokens_per_minute)
            wait_time = max(wait_time, wait_time_input)
        
        # Prüfe Output-Token-Limit
        if recent_output_tokens + output_tokens_estimate > self.output_tokens_per_minute:
            wait_time_output = self._calculate_token_wait_time(self.output_token_usage, 
                                                             output_tokens_estimate, 
                                                             self.output_tokens_per_minute)
            wait_time = max(wait_time, wait_time_output)
        
        # Warte, wenn nötig
        if wait_time > 0:
            print(f"Rate-Limit erreicht. Warte {wait_time:.1f} Sekunden...")
            time.sleep(wait_time)
            # Nach dem Warten die abgelaufenen Einträge erneut bereinigen
            current_time = time.time()
            self._clean_expired_entries(current_time)
    
    def _clean_expired_entries(self, current_time):
        """Entfernt Einträge, die älter als 60 Sekunden sind."""
        cutoff_time = current_time - 60
        
        # Bereinige Request-Timestamps
        while self.request_timestamps and self.request_timestamps[0] < cutoff_time:
            self.request_timestamps.pop(0)
        
        # Bereinige Input-Token-Nutzung
        while self.input_token_usage and self.input_token_usage[0][0] < cutoff_time:
            self.input_token_usage.pop(0)
        
        # Bereinige Output-Token-Nutzung
        while self.output_token_usage and self.output_token_usage[0][0] < cutoff_time:
            self.output_token_usage.pop(0)
    
    def _calculate_token_wait_time(self, token_usage, new_tokens, limit):
        """Berechnet die Wartezeit für Token-basierte Limits."""
        if not token_usage:
            return 0
            
        current_usage = sum(tokens for _, tokens in token_usage)
        
        # Wenn das Hinzufügen neuer Tokens das Limit überschreitet, müssen wir warten
        if current_usage + new_tokens > limit:
            # Berechne, wie viele Token wir freigeben müssen
            tokens_to_free = (current_usage + new_tokens) - limit
            
            # Finde den Zeitpunkt, zu dem genügend Token freigegeben werden
            freed_tokens = 0
            for timestamp, tokens in token_usage:
                freed_tokens += tokens
                if freed_tokens >= tokens_to_free:
                    # Warte bis dieser Zeitstempel + 60 Sekunden
                    return timestamp + 60 - time.time()
        
        return 0
    
    def record_usage(self, input_tokens, output_tokens):
        """Zeichnet die Nutzung einer API-Anfrage auf."""
        current_time = time.time()
        self.request_timestamps.append(current_time)
        self.input_token_usage.append([current_time, input_tokens])
        self.output_token_usage.append([current_time, output_tokens])

# Erstelle eine globale Rate Limiter-Instanz
rate_limiter = RateLimiter()

def estimate_token_count(text):
    """
    Schätzt die Anzahl der Tokens im Text. 
    Einfache Approximation: 1 Token ≈ 4 Zeichen für Englisch/Deutsch.
    """
    return len(text) // 4

def generate_synthetic_prompt(court_ruling, press_release, model="claude-3-7-sonnet-20250219", retries=3, wait_time=2):
    """
    Generiert einen synthetischen Prompt, der die gegebene Pressemitteilung aus dem Gerichtsurteil erzeugen könnte.
    
    Args:
        court_ruling (str): Der Text des Gerichtsurteils
        press_release (str): Der Text der Pressemitteilung
        model (str): Das zu verwendende Claude-Modell
        retries (int): Anzahl der Wiederholungsversuche bei API-Fehlern
        wait_time (int): Wartezeit zwischen Wiederholungsversuchen in Sekunden
        
    Returns:
        str: Der generierte synthetische Prompt
    """
    # Vorbereitung des System-Prompts
    system_prompt = """
    Du bist ein Experte für juristische Texte und Kommunikation. Deine Aufgabe ist es, ein Gerichtsurteil und die 
    dazugehörige Pressemitteilung zu analysieren und dann herauszufinden, welcher Prompt verwendet worden sein könnte, 
    um diese Pressemitteilung aus dem Gerichtsurteil zu generieren, wenn man ihn einem LLM gegeben hätte.
    
    1. Analysiere, wie die Pressemitteilung Informationen aus dem Urteil vereinfacht, umstrukturiert und Schlüsselinformationen hervorhebt
    2. Berücksichtige den Ton, die Struktur und den Detaillierungsgrad der Pressemitteilung
    3. Identifiziere, welche Anweisungen nötig wären, um den juristischen Text in diese Pressemitteilung zu transformieren
    
    Erkläre NICHT deine Überlegungen und füge KEINE Meta-Kommentare hinzu. Gib NUR den tatsächlichen Prompt aus, der die 
    Pressemitteilung aus dem Gerichtsurteil generieren würde. Sei spezifisch und detailliert in deinem synthetisierten Prompt.
    """
    
    # Vorbereitung des Benutzer-Prompts
    user_prompt = f"""
    Hier ist das originale Gerichtsurteil:
    
    ```
    {court_ruling}
    ```
    
    Und hier ist die Pressemitteilung, die daraus erstellt wurde:
    
    ```
    {press_release}
    ```
    
    Erstelle einen detaillierten Prompt, der einem LLM gegeben werden könnte, um die obige Pressemitteilung aus dem Gerichtsurteil zu generieren. 
    Schreibe NUR den Prompt selbst, ohne Erklärungen oder Meta-Kommentare.
    """
    
    # Schätze den Token-Verbrauch
    input_tokens_estimate = estimate_token_count(system_prompt) + estimate_token_count(user_prompt)
    output_tokens_estimate = 500  # Konservative Schätzung für die Antwortlänge
    
    # Warte, wenn nötig, um Rate-Limits einzuhalten
    rate_limiter.wait_if_needed(input_tokens_estimate, output_tokens_estimate)
    
    for attempt in range(retries):
        try:
            response = client.messages.create(
                model=model,
                max_tokens=1000,
                system=system_prompt,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            
            # Zeichne tatsächliche Token-Nutzung auf, falls verfügbar
            input_tokens = response.usage.input_tokens if hasattr(response, 'usage') and hasattr(response.usage, 'input_tokens') else input_tokens_estimate
            output_tokens = response.usage.output_tokens if hasattr(response, 'usage') and hasattr(response.usage, 'output_tokens') else output_tokens_estimate
            rate_limiter.record_usage(input_tokens, output_tokens)
            
            return response.content[0].text.strip()
        except Exception as e:
            if attempt < retries - 1:
                print(f"Fehler: {e}. Neuer Versuch in {wait_time} Sekunden...")
                time.sleep(wait_time)
            else:
                print(f"Nach {retries} Versuchen fehlgeschlagen: {e}")
                return f"Fehler bei der Generierung des Prompts: {e}"

def process_batch(df, batch_size=5, start_idx=0, save_interval=10, fix_errors=False):
    """
    Verarbeitet einen Dataframe in Batches und generiert synthetische Prompts für jede Zeile.
    Der Batch-Size wurde auf 5 reduziert, um die Rate-Limits besser einzuhalten.
    
    Args:
        df (pd.DataFrame): Der Dataframe mit Gerichtsurteilen und Pressemitteilungen
        batch_size (int): Anzahl der Elemente, die vor dem Speichern von Zwischenergebnissen verarbeitet werden
        start_idx (int): Index, bei dem die Verarbeitung beginnen soll (für die Wiederaufnahme)
        save_interval (int): Wie oft Zwischenergebnisse gespeichert werden sollen
        fix_errors (bool): Wenn True, werden Zeilen mit API-Fehlermeldungen erneut verarbeitet
        
    Returns:
        pd.DataFrame: Der Dataframe mit hinzugefügten synthetischen Prompts
    """
    # Erstelle eine synthetic_prompt-Spalte, falls sie nicht existiert
    if 'synthetic_prompt' not in df.columns:
        df['synthetic_prompt'] = None
    
    # Identifiziere Zeilen mit API-Fehlermeldungen, falls fix_errors=True
    if fix_errors:
        error_mask = df['synthetic_prompt'].astype(str).str.contains("Fehler bei der Generierung des Prompts", na=False)
        error_indices = df[error_mask].index.tolist()
        if error_indices:
            print(f"Gefunden: {len(error_indices)} Einträge mit API-Fehlern")
            # Setze die fehlerhaften Einträge auf None zurück, damit sie neu verarbeitet werden
            df.loc[error_indices, 'synthetic_prompt'] = None
    
    # Erstelle ein Verzeichnis für Checkpoints, falls es nicht existiert
    checkpoint_dir = Path('checkpoints')
    checkpoint_dir.mkdir(exist_ok=True)
    
    # Verarbeite in Batches
    for i in tqdm(range(start_idx, len(df), batch_size)):
        batch_end = min(i + batch_size, len(df))
        batch = df.iloc[i:batch_end].copy()
        
        # Zähler für erfolgreiche Verarbeitungen in diesem Batch
        successful_in_batch = 0
        
        for idx, row in batch.iterrows():
            if pd.isna(df.at[idx, 'synthetic_prompt']):
                court_ruling = row['judgement']
                press_release = row['summary']
                
                # Kürze sehr lange Eingaben, um Token-Limits zu vermeiden
                if len(court_ruling) > 12000:  # Reduziert für bessere Tokenbegrenzung
                    court_ruling = court_ruling[:12000] + "..."
                if len(press_release) > 4000:  # Reduziert für bessere Tokenbegrenzung
                    press_release = press_release[:4000] + "..."
                
                # Generiere den synthetischen Prompt
                synthetic_prompt = generate_synthetic_prompt(court_ruling, press_release)
                
                # Prüfe, ob die Antwort einen API-Fehler enthält
                if "Fehler bei der Generierung des Prompts" in str(synthetic_prompt):
                    print(f"⚠️ API-Fehler bei Index {idx}, wird in einem zukünftigen Durchlauf erneut versucht")
                else:
                    successful_in_batch += 1
                
                df.at[idx, 'synthetic_prompt'] = synthetic_prompt
                
                # Zeige Fortschritt an
                if (idx - i) % 5 == 0 or idx == batch_end - 1:
                    print(f"{idx+1}/{len(df)} Einträge verarbeitet - {successful_in_batch}/{batch_end-i} erfolgreich in diesem Batch")
        
        # Speichere Checkpoint in regelmäßigen Abständen
        if (i // batch_size) % save_interval == 0 or batch_end == len(df):
            checkpoint_path = checkpoint_dir / f"synthetic_prompts_checkpoint_{batch_end}.csv"
            df.to_csv(checkpoint_path, index=False)
            print(f"Checkpoint gespeichert unter {checkpoint_path}")
            
            # Kurze Pause nach jedem Batch, um Rate-Limits zu entspannen
            time.sleep(1)
    
    return df

## 4. Checkpoint laden oder von vorne beginnen

Wir prüfen, ob es eine Checkpoint-Datei gibt, um von dort fortzufahren, oder beginnen die Verarbeitung von Anfang an.

In [45]:
# Prüfe auf vorhandene Checkpoints
checkpoint_dir = Path('checkpoints')
checkpoint_files = list(checkpoint_dir.glob("synthetic_prompts_checkpoint_*.csv")) if checkpoint_dir.exists() else []

start_idx = 0

if checkpoint_files:
    # Finde den neuesten Checkpoint basierend auf der Nummer im Dateinamen
    latest_checkpoint = max(checkpoint_files, key=lambda x: int(x.stem.split('_')[-1]))
    print(f"Neuester Checkpoint gefunden: {latest_checkpoint}")
    
    # Frage, ob vom Checkpoint fortgefahren werden soll
    continue_from_checkpoint = input("Vom letzten Checkpoint fortfahren? (j/n): ").lower().strip() == 'j'
    
    if continue_from_checkpoint:
        # Lade den Checkpoint
        df = pd.read_csv(latest_checkpoint)
        print(f"Checkpoint mit {len(df)} Einträgen geladen")
        
        # Finde die erste Zeile ohne synthetischen Prompt
        if 'synthetic_prompt' in df.columns:
            missing_prompts = df['synthetic_prompt'].isna()
            if missing_prompts.any():
                start_idx = missing_prompts.idxmax()
                print(f"Verarbeitung wird ab Index {start_idx} fortgesetzt")
            else:
                print("Alle Einträge haben bereits synthetische Prompts!")
        else:
            print("Keine synthetic_prompt-Spalte im Checkpoint gefunden, beginne von vorne")
    else:
        print("Starte neue Verarbeitung")

Neuester Checkpoint gefunden: checkpoints/synthetic_prompts_checkpoint_560.csv
Checkpoint mit 6432 Einträgen geladen
Verarbeitung wird ab Index 560 fortgesetzt


## Fehlerhafte Einträge korrigieren

In diesem Abschnitt identifizieren und korrigieren wir Einträge, bei denen die API mit einem Authentifizierungsfehler geantwortet hat.

In [51]:
# Lade den zuletzt erstellten Checkpoint, falls vorhanden
checkpoint_dir = Path('checkpoints')
checkpoint_files = list(checkpoint_dir.glob("synthetic_prompts_checkpoint_*.csv")) if checkpoint_dir.exists() else []

if checkpoint_files:
    # Finde den neuesten Checkpoint basierend auf der Nummer im Dateinamen
    latest_checkpoint = max(checkpoint_files, key=lambda x: int(x.stem.split('_')[-1]))
    print(f"Neuester Checkpoint geladen: {latest_checkpoint}")
    
    # Lade den Checkpoint
    fix_df = pd.read_csv(latest_checkpoint)
    print(f"Checkpoint mit {len(fix_df)} Einträgen geladen")
    
    # Identifiziere Zeilen mit API-Fehlern - sucht nach dem generellen Fehlerpattern
    error_mask = fix_df['synthetic_prompt'].astype(str).str.contains("Fehler bei der Generierung des Prompts", na=False)
    error_indices = fix_df[error_mask].index.tolist()
    
    if error_indices:
        print(f"Gefunden: {len(error_indices)} Einträge mit API-Fehlern")
        print(f"Erste 5 Indices mit Fehlern: {error_indices[:5]}")
        print(f"Beispiel-Fehlermeldung: {fix_df.loc[error_indices[0], 'synthetic_prompt']}")
        
        # Option zum Korrigieren aller fehlerhaften Einträge
        fix_all = input("Alle fehlerhaften Einträge korrigieren? (j/n): ").lower().strip() == 'j'
        
        if fix_all:
            print("Korrigiere fehlerhafte Einträge...")
            # Verarbeite den Datensatz und korrigiere fehlerhafte Einträge
            fix_df = process_batch(fix_df, batch_size=5, start_idx=0, save_interval=2, fix_errors=True)
            
            # Prüfe, ob noch Fehler vorhanden sind
            remaining_errors = fix_df['synthetic_prompt'].astype(str).str.contains("Fehler bei der Generierung des Prompts", na=False).sum()
            print(f"Verbleibende Fehler nach Korrektur: {remaining_errors}")
            
            # Speichere das Ergebnis mit korrigierten Einträgen
            fixed_output_path = checkpoint_dir / f"synthetic_prompts_fixed_{int(time.time())}.csv"
            fix_df.to_csv(fixed_output_path, index=False)
            print(f"Korrigierter Datensatz gespeichert unter {fixed_output_path}")
        else:
            print("Korrektur abgebrochen.")
    else:
        print("Keine Einträge mit API-Fehlern gefunden.")
else:
    print("Keine Checkpoints gefunden. Bitte führe zunächst die Generierung der synthetischen Prompts durch.")

Neuester Checkpoint geladen: checkpoints/synthetic_prompts_checkpoint_580.csv
Checkpoint mit 6432 Einträgen geladen
Gefunden: 540 Einträge mit API-Fehlern
Erste 5 Indices mit Fehlern: [0, 1, 2, 3, 4]
Beispiel-Fehlermeldung: Fehler bei der Generierung des Prompts: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"
Korrektur abgebrochen.


## 5. Synthetische Prompts generieren

Jetzt verarbeiten wir den Datensatz, um die synthetischen Prompts zu generieren.

In [ ]:
# Konfiguriere Parameter für die Batch-Verarbeitung
batch_size = 5  # Reduziert von 10 auf 5 für bessere Rate-Limiting
save_interval = 5  # Speichere Checkpoint alle 5 Batches

# Verarbeite den Datensatz
print(f"Starte Generierung synthetischer Prompts für {len(df)} Einträge ab Index {start_idx}")
print(f"Rate-Limits: 50 Anfragen/Minute, 20.000 Input-Tokens/Minute, 8.000 Output-Tokens/Minute")
df = process_batch(df, batch_size=batch_size, start_idx=start_idx, save_interval=save_interval)

# Speichere das endgültige Ergebnis
output_path = Path('../data/processed/court_press_with_synthetic_prompts.csv')
df.to_csv(output_path, index=False)
print(f"Vollständiger Datensatz mit synthetischen Prompts gespeichert unter {output_path}")

## 6. Beispiele der generierten synthetischen Prompts analysieren

Schauen wir uns einige Beispiele der generierten synthetischen Prompts an, um die Art der Anweisungen zu verstehen, die Claude vorgeschlagen hat.

In [ ]:
# Wähle einige Beispiele zur Anzeige aus
if 'synthetic_prompt' in df.columns and not df['synthetic_prompt'].isna().all():
    # Hole 5 zufällige Beispiele, bei denen synthetic_prompt nicht None/NaN ist
    sample_df = df[df['synthetic_prompt'].notna()].sample(min(5, len(df[df['synthetic_prompt'].notna()])))
    
    for i, (_, row) in enumerate(sample_df.iterrows(), 1):
        print(f"\n==== Beispiel {i} ====")
        print(f"Gerichtsfall-ID: {row['id']}")
        print("\nSynthetischer Prompt:")
        print(row['synthetic_prompt'])
        
        # Optional, zeige auch den ersten Teil der Pressemitteilung
        print("\nErste 150 Zeichen der Pressemitteilung:")
        print(row['summary'][:150] + "...")
        print("-" * 80)
else:
    print("Noch keine synthetischen Prompts verfügbar. Führe zunächst die Verarbeitungszelle aus.")

## 7. Muster in den synthetischen Prompts analysieren

Analysieren wir die generierten Prompts, um häufige Muster und Eigenschaften zu identifizieren.

In [ ]:
# Prüfe, ob wir Prompts zur Analyse haben
if 'synthetic_prompt' in df.columns and not df['synthetic_prompt'].isna().all():
    # Erstelle einen Dataframe nur mit den synthetischen Prompts für die Analyse
    prompts_df = df[df['synthetic_prompt'].notna()].copy()
    
    # Berechne grundlegende Statistiken
    prompt_lengths = prompts_df['synthetic_prompt'].str.len()
    avg_length = prompt_lengths.mean()
    min_length = prompt_lengths.min()
    max_length = prompt_lengths.max()
    
    print(f"Anzahl synthetischer Prompts: {len(prompts_df)}")
    print(f"Durchschnittliche Prompt-Länge: {avg_length:.1f} Zeichen")
    print(f"Kürzester Prompt: {min_length} Zeichen")
    print(f"Längster Prompt: {max_length} Zeichen")
    
    # Suche nach häufigen Phrasen oder Anweisungstypen in den Prompts
    common_phrases = [
        "zusammenfassen", "vereinfachen", "erklären", "übersetzen", "umwandeln", 
        "transformieren", "erstelle eine pressemitteilung", "schreibe eine pressemitteilung", 
        "nicht-technisch", "laien", "öffentliches publikum", "prägnant"
    ]
    
    phrase_counts = {}
    for phrase in common_phrases:
        count = prompts_df['synthetic_prompt'].str.lower().str.contains(phrase).sum()
        if count > 0:
            phrase_counts[phrase] = count
    
    # Konvertiere Anzahl in Prozentsätze
    phrase_percentages = {k: v/len(prompts_df)*100 for k, v in phrase_counts.items()}
    
    # Sortiere nach Häufigkeit
    sorted_phrases = sorted(phrase_percentages.items(), key=lambda x: x[1], reverse=True)
    
    print("\nHäufige Anweisungstypen in synthetischen Prompts:")
    for phrase, percentage in sorted_phrases:
        print(f"{phrase}: {percentage:.1f}% der Prompts")
else:
    print("Noch keine synthetischen Prompts verfügbar. Führe zunächst die Verarbeitungszelle aus.")

## 8. Datensatz für weitere Analysen exportieren

Schließlich speichern wir unseren Datensatz mit den synthetischen Prompts für weitere Analysen.

In [43]:
# Erstelle eine endgültige Version des Datensatzes mit Spalten in einer logischen Reihenfolge
if 'synthetic_prompt' in df.columns:
    # Reorganisiere Spalten in einer logischen Reihenfolge
    columns_order = [
        'id', 'date', 'subset_name', 'split_name',  # Kennungen
        'synthetic_prompt',  # Neue generierte Spalte
        'summary', 'judgement'  # Ursprüngliche Textspalten
    ]
    
    # Füge alle verbleibenden Spalten hinzu
    remaining_columns = [col for col in df.columns if col not in columns_order]
    final_columns = columns_order + remaining_columns
    
    # Erstelle endgültigen Datensatz mit Spalten in gewünschter Reihenfolge
    final_df = df[final_columns].copy()
    
    # Speichere als endgültigen Datensatz
    final_output_path = Path('../data/processed/court_press_with_synthetic_prompts_final.csv')
    final_df.to_csv(final_output_path, index=False)
    print(f"Endgültiger Datensatz mit {len(final_df)} Einträgen gespeichert unter {final_output_path}")
    
    # Speichere auch eine leichtere Version mit nur den wesentlichen Spalten
    essential_columns = ['id', 'date', 'subset_name', 'split_name', 'synthetic_prompt', 'summary', 'judgement']
    essential_df = final_df[essential_columns].copy()
    
    essential_output_path = Path('../data/processed/court_press_with_synthetic_prompts_essential.csv')
    essential_df.to_csv(essential_output_path, index=False)
    print(f"Wesentlicher Datensatz mit {len(essential_df)} Einträgen gespeichert unter {essential_output_path}")
else:
    print("Noch keine synthetischen Prompts verfügbar. Führe zunächst die Verarbeitungszelle aus.")

Endgültiger Datensatz mit 6432 Einträgen gespeichert unter ../data/processed/court_press_with_synthetic_prompts_final.csv
Wesentlicher Datensatz mit 6432 Einträgen gespeichert unter ../data/processed/court_press_with_synthetic_prompts_essential.csv
